In [1]:
import pandas as pd
import geopandas as gpd

In [2]:
# don't update these that have been locked
lstLockForecastArea = ['WFRC', 'MAG', 'Summit', 'Iron', 'Dixie', 'Cache']

# GLOBAL VARIABLES

dfObsVolSource = pd.DataFrame([
    ['AADTHistory_2023.xlsx'             , [0]      ],
    ['AADTHistory.xlsx'                  , [2,3,4,5]],
    ['WF_Segments_20240326_Draft.shp'    , [1]      ]
],columns=('SOURCE','SUBAREAID'))

dfObsVolSource = dfObsVolSource.explode('SUBAREAID')

dfObsVolSource.to_csv('intermediate/obsvolsource.csv',index=False)

# base year adjustment
# segment shapefile with AADT and previous forecasts - copied from 'A:/1 - TDM/3 - Model Dev/1 - WF/1 - Official Release/v9x/v9.0/WF TDM v9.0 - official/1_Inputs/6_Segment/Segments_WF - 2023-08-01.shp'
baseYear = 2019

dfFactorSource = pd.DataFrame([
    ['Segments_State_20231221_Draft.shp' , [0,2,3,4,5]],
    ['WF_Segments_20240326_Draft.shp'    , [1]        ]
],columns=('SOURCE','SUBAREAID'))

dfFactorSource = dfFactorSource.explode('SUBAREAID')

# factors file from last notebook
fnFactors = 'intermediate/factors.csv'

# field for weekday factor
colFacWdAvg = 'FAC_WDAVG'
colFacSpr   = 'FAC_SPR'
colFacFal   = 'FAC_FAL'
# SHOULD WE ADJUST WEEKDAY FACTOR INTO FUTURE?? 

# some models are AWDT for whole year and some are AADT for a season
# this list of SUBAREAIDs is for models that are AADT for season and need to be adjusted by a Spring/Fall factor
subareaidsThatUseSprFalFac = [4]

# where the same segid in multiple subareas and model, drop order for duplicate segids... pick order to drop from models
duplicateSegsInTdms = pd.DataFrame([
    ['0015_042.2', 3],
    ['0065_002.4', 0],
    ['0150_029.4', 0],
    ['0150_030.0', 0],
    ['0150_033.3', 0],
    ['1822_000.0', 0],
    ['2863_000.0', 4],
    ['2865_019.4', 4]
], columns=('SEGID','removeSUBAREAID'))

duplicateSegsInTdms.to_csv('intermediate/duplicateSegsInTdms.csv',index=False)

# where the same segid in multiple subareas and model, drop order for duplicate segids... pick order to drop from models
hardCodeSegIds = pd.DataFrame([
    ['0006_222.1', 0]
], columns=('SEGID','hardCodeSUBAREAID'))

hardCodeSegIds.to_csv('intermediate/hardCodeSegIds.csv',index=False)

dfModelSegSummaries = pd.DataFrame( [
    [0, 2019, 'data/model-output/0_USTM - v3.0 - RELEASED/_Summary_SEGID_BY2019.csv'           ],
    [0, 2023, 'data/model-output/0_USTM - v3.0 - RELEASED/_Summary_SEGID_OY2023.csv'           ],
    [0, 2028, 'data/model-output/0_USTM - v3.0 - RELEASED/_Summary_SEGID_STIP2028.csv'         ],
    [0, 2032, 'data/model-output/0_USTM - v3.0 - RELEASED/_Summary_SEGID_LRP2032.csv'          ],
    [0, 2042, 'data/model-output/0_USTM - v3.0 - RELEASED/_Summary_SEGID_LRP2042.csv'          ],
    [0, 2050, 'data/model-output/0_USTM - v3.0 - RELEASED/_Summary_SEGID_LRP2050.csv'          ],
    [1, 2019, 'data/model-output/1_WF/v9_SE19_Net19_Summary_SEGID.csv'                         ],
    [1, 2023, 'data/model-output/1_WF/v9_SE23_Net23_Summary_SEGID.csv'                         ],
    [1, 2028, 'data/model-output/1_WF/v9_TIP_SE28_Net28_Summary_SEGID.csv'                     ],
    [1, 2032, 'data/model-output/1_WF/v9_RTP_SE32_Net32_Summary_SEGID.csv'                     ],
    [1, 2042, 'data/model-output/1_WF/v9_RTP_SE42_Net42_Summary_SEGID.csv'                     ],
    [1, 2050, 'data/model-output/1_WF/v9_RTP_SE50_Net50_Summary_SEGID.csv'                     ],
    [2, 2019, 'data/model-output/2_Cache/BY2019_Summary_SEGID.dbf'                             ],
    [2, 2023, 'data/model-output/2_Cache/BY2023_Summary_SEGID.dbf'                             ],
    [2, 2028, 'data/model-output/2_Cache/Summary_SEGID_STIP28.dbf'                             ],
    [2, 2032, 'data/model-output/2_Cache/Summary_SEGID_LRP23_32.dbf'                           ],
    [2, 2042, 'data/model-output/2_Cache/Summary_SEGID_LRP23_42.dbf'                           ],
    [2, 2050, 'data/model-output/2_Cache/Summary_SEGID_LRP23_50.dbf'                           ],
    [3, 2019, 'data/model-output/3_Dixie/Summary_SEGID_19.dbf'                                 ],
    [3, 2023, 'data/model-output/3_Dixie/Summary_SEGID_23.dbf'                                 ],
    [3, 2028, 'data/model-output/3_Dixie/Summary_SEGID_28.dbf'                                 ],
    [3, 2032, 'data/model-output/3_Dixie/Summary_SEGID_32.dbf'                                 ],
    [3, 2042, 'data/model-output/3_Dixie/Summary_SEGID_42.dbf'                                 ],
    [3, 2050, 'data/model-output/3_Dixie/Summary_SEGID_50.dbf'                                 ],
    [4, 2019, 'data/model-output/4_SuWsv2_2024-02-08_DRAFT/Summary_SEGID_BY2019.dbf'           ],
    [4, 2023, 'data/model-output/4_SuWsv2_2024-02-08_DRAFT/Summary_SEGID_OY2023.dbf'           ],
    [4, 2028, 'data/model-output/4_SuWsv2_2024-02-08_DRAFT/Summary_SEGID_STIP2028.dbf'         ],
    [4, 2032, 'data/model-output/4_SuWsv2_2024-02-08_DRAFT/Summary_SEGID_LRP2032.dbf'          ],
    [4, 2042, 'data/model-output/4_SuWsv2_2024-02-08_DRAFT/Summary_SEGID_LRP2042.dbf'          ],
    [4, 2050, 'data/model-output/4_SuWsv2_2024-02-08_DRAFT/Summary_SEGID_LRP2050.dbf'          ],
    [5, 2019, 'data/model-output/5_IronCo - v1.0 - 2023-09-13_DRAFT/Summary_SEGID_BY2019.dbf'  ],
    [5, 2023, 'data/model-output/5_IronCo - v1.0 - 2023-09-13_DRAFT/Summary_SEGID_OY2023.dbf'  ],
    [5, 2028, 'data/model-output/5_IronCo - v1.0 - 2023-09-13_DRAFT/Summary_SEGID_STIP2028.dbf'],
    [5, 2032, 'data/model-output/5_IronCo - v1.0 - 2023-09-13_DRAFT/Summary_SEGID_FY2032.dbf'  ],
    [5, 2042, 'data/model-output/5_IronCo - v1.0 - 2023-09-13_DRAFT/Summary_SEGID_FY2042.dbf'  ],
    [5, 2050, 'data/model-output/5_IronCo - v1.0 - 2023-09-13_DRAFT/Summary_SEGID_FY2050.dbf'  ]
], columns=('modSubareaId','modYear','modSegSummaryFile'))

# seg summary fields to be useds
use_fields = ['SEGID','SUBAREAID','DY_VOL','FT','ATYPENAME','LANES']

# drop the following records that have following SEGIDs
dropSegId = ['NO','N0']

#display(dfModelSegSummaries)


In [3]:
# Required libraries
from dbfread import DBF
import pandas as pd

# Create a list to store DataFrames read from each file
frames = []

# Iterate through the rows and read each file
for index, row in dfModelSegSummaries.iterrows():
    # Prepend path with 'dbfs:/'
    file_path = row['modSegSummaryFile']
    print(file_path)
    
    # Check the file extension
    if file_path.endswith('.dbf'):
        # Read the .dbf file with dbfread
        table = DBF(file_path)
        df = pd.DataFrame(iter(table))  # Convert the DBF table to a DataFrame
    elif file_path.endswith('.csv'):
        df = pd.read_csv(file_path, low_memory=False)
    else:
        # If the file is neither .csv nor .dbf, skip this iteration
        print(f"Unsupported file format for file: {file_path}")
        continue

    df = df[use_fields]

    # Update SUBAREAID in df where there's a matching SEGID
    mapping = hardCodeSegIds.set_index('SEGID')['hardCodeSUBAREAID']
    df['SUBAREAID'] = df['SEGID'].map(mapping).fillna(df['SUBAREAID'])

    # filter out where link SUBAREAID match the model SUBAREAID... Mostly for USTM that spans multiple model spaces
    df = df[df['SUBAREAID']==row['modSubareaId']]

    # drop SEGIDs defined in dropSegId list
    df = df[~df['SEGID'].isin(dropSegId)]

    df['YEAR'] = row['modYear'] # Add modYear column

    df['inTdm'] = 1 # set to 1 for segments in TDM... so we can tell what isn't in TDM later

    frames.append(df)

# Concatenate all the frames into a single DataFrame
dfMdlVol = pd.concat(frames, ignore_index=True)

# If you're using Databricks, use 'display' to show the DataFrame
display(dfMdlVol)

data/model-output/0_USTM - v3.0 - RELEASED/_Summary_SEGID_BY2019.csv
data/model-output/0_USTM - v3.0 - RELEASED/_Summary_SEGID_OY2023.csv
data/model-output/0_USTM - v3.0 - RELEASED/_Summary_SEGID_STIP2028.csv
data/model-output/0_USTM - v3.0 - RELEASED/_Summary_SEGID_LRP2032.csv
data/model-output/0_USTM - v3.0 - RELEASED/_Summary_SEGID_LRP2042.csv
data/model-output/0_USTM - v3.0 - RELEASED/_Summary_SEGID_LRP2050.csv
data/model-output/1_WF/v9_SE19_Net19_Summary_SEGID.csv
data/model-output/1_WF/v9_SE23_Net23_Summary_SEGID.csv
data/model-output/1_WF/v9_TIP_SE28_Net28_Summary_SEGID.csv
data/model-output/1_WF/v9_RTP_SE32_Net32_Summary_SEGID.csv
data/model-output/1_WF/v9_RTP_SE42_Net42_Summary_SEGID.csv
data/model-output/1_WF/v9_RTP_SE50_Net50_Summary_SEGID.csv
data/model-output/2_Cache/BY2019_Summary_SEGID.dbf
data/model-output/2_Cache/BY2023_Summary_SEGID.dbf
data/model-output/2_Cache/Summary_SEGID_STIP28.dbf
data/model-output/2_Cache/Summary_SEGID_LRP23_32.dbf
data/model-output/2_Cache/Sum

,SEGID,SUBAREAID,DY_VOL,FT,ATYPENAME,LANES,YEAR,inTdm
0,0006_000.0,0.0,126.5,2.0,Rural,1.0,2019,1
1,0006_000.7,0.0,216.6,2.0,Rural,1.0,2019,1
2,0006_016.0,0.0,79.9,2.0,Rural,1.0,2019,1
3,0006_046.0,0.0,79.9,2.0,Rural,1.0,2019,1
4,0006_060.2,0.0,83.6,2.0,Rural,1.0,2019,1
...,...,...,...,...,...,...,...,...
49888,3218_006.4,5.0,49.5,7.0,Rural,2.0,2050,1
49889,UDOT_7006,5.0,4181.0,4.0,Transition,4.0,2050,1
49890,UDOT_7007,5.0,7259.0,4.0,Rural,4.0,2050,1
49891,UDOT_7008,5.0,453.9,4.0,Rural,2.0,2050,1


# Add Factors

In [4]:
# read intermediate CSV with base year observed AADT


In [5]:
dfObsVolSource

,SOURCE,SUBAREAID
0,AADTHistory_2023.xlsx,0
1,AADTHistory.xlsx,2
1,AADTHistory.xlsx,3
1,AADTHistory.xlsx,4
1,AADTHistory.xlsx,5
2,WF_Segments_20240326_Draft.shp,1


In [6]:
gdfSegments = pd.read_csv('intermediate/segments.gdf',low_memory=False)

# use cofips to match to segments that don't have factors

dfSegsForFacSource = pd.merge(gdfSegments[['SEGID','CO_FIPS','SUBAREAID','SOURCE']],dfFactorSource, on=('SUBAREAID','SOURCE'))

dfSegsForFacSource.fillna(0,inplace=True)

dfSegsForFacSource = dfSegsForFacSource[~dfSegsForFacSource['SEGID'].str.contains('UTA_')]

dfSegsForFacSource.groupby(['SEGID']).agg(COUNT=('SOURCE','count'))

,COUNT
SEGID,
0006_000.0,1
0006_000.7,1
0006_016.0,1
0006_046.0,1
0006_060.2,1
...,...
WFRC_8472,1
WFRC_8473,1
WFRC_8474,1


In [7]:

# Create a DataFrame with all unique years

all_years_df = pd.DataFrame(dfMdlVol['YEAR'].unique(), columns=['YEAR'])
display(all_years_df)


# Merge dfSegsForFacSource with all_years_df to create all combinations of SEGID and YEAR
dfSegsForFacSource_AllYears = pd.merge(dfSegsForFacSource.assign(key=1), all_years_df.assign(key=1), on='key').drop('key', axis=1)


# Correcting the syntax for renaming columns and merging
dfMdlVolWithAllSegs = pd.merge(
    dfSegsForFacSource_AllYears,#.rename(columns={'SUBAREAID': 'SAID_FAC'}), 
    dfMdlVol, 
    on=('SEGID','YEAR','SUBAREAID'),
    how='left'
)


# FILL SEGS THAT DON'T HAVE MODEL WITH BASIC ASSUMPTIONS
dfMdlVolWithAllSegs['inTdm'].fillna(0, inplace=True)
dfMdlVolWithAllSegs['DY_VOL'].fillna(1, inplace=True) # since not in any model year, but should have volume so give at least one... since 0 volume segments in model should have volume go away
dfMdlVolWithAllSegs.fillna(0, inplace=True)
dfMdlVolWithAllSegs

,YEAR
0,2019
1,2023
2,2028
3,2032
4,2042
5,2050


,SEGID,CO_FIPS,SUBAREAID,SOURCE,YEAR,DY_VOL,FT,ATYPENAME,LANES,inTdm
0,0006_000.0,27.0,0.0,Segments_State_20231221_Draft.shp,2019,126.5,2.0,Rural,1.0,1.0
1,0006_000.0,27.0,0.0,Segments_State_20231221_Draft.shp,2023,99.7,2.0,Rural,1.0,1.0
2,0006_000.0,27.0,0.0,Segments_State_20231221_Draft.shp,2028,101.7,2.0,Rural,1.0,1.0
3,0006_000.0,27.0,0.0,Segments_State_20231221_Draft.shp,2032,103.8,2.0,Rural,1.0,1.0
4,0006_000.0,27.0,0.0,Segments_State_20231221_Draft.shp,2042,108.1,2.0,Rural,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...
57367,WFRC_8430,35.0,1.0,WF_Segments_20240326_Draft.shp,2023,6392.6,3.0,Suburban,2.0,1.0
57368,WFRC_8430,35.0,1.0,WF_Segments_20240326_Draft.shp,2028,7340.4,3.0,Suburban,2.0,1.0
57369,WFRC_8430,35.0,1.0,WF_Segments_20240326_Draft.shp,2032,6968.0,3.0,Suburban,2.0,1.0
57370,WFRC_8430,35.0,1.0,WF_Segments_20240326_Draft.shp,2042,8527.4,3.0,Suburban,2.0,1.0


In [8]:
dfMdlVolWithAllSegs[dfMdlVolWithAllSegs['SEGID']=='1695_000.0']

,SEGID,CO_FIPS,SUBAREAID,SOURCE,YEAR,DY_VOL,FT,ATYPENAME,LANES,inTdm
10098,1695_000.0,19.0,0.0,Segments_State_20231221_Draft.shp,2019,1558.0,4.0,Transition,1.0,1.0
10099,1695_000.0,19.0,0.0,Segments_State_20231221_Draft.shp,2023,1515.1,4.0,Transition,1.0,1.0
10100,1695_000.0,19.0,0.0,Segments_State_20231221_Draft.shp,2028,1673.5,4.0,Transition,1.0,1.0
10101,1695_000.0,19.0,0.0,Segments_State_20231221_Draft.shp,2032,1787.5,4.0,Transition,1.0,1.0
10102,1695_000.0,19.0,0.0,Segments_State_20231221_Draft.shp,2042,1959.2,4.0,Transition,1.0,1.0
10103,1695_000.0,19.0,0.0,Segments_State_20231221_Draft.shp,2050,1991.5,4.0,Transition,1.0,1.0


In [9]:
# GET WEEKDAY FACTORS

# read in factors
_dfFac = pd.read_csv(fnFactors, usecols=('SOURCE', 'SEGID', colFacWdAvg, colFacSpr, colFacFal))

_dfFac['FAC_SPRFAL'] = (_dfFac[colFacSpr] + _dfFac[colFacFal]) / 2

dfMdlVolWithFac = pd.merge(dfMdlVolWithAllSegs, _dfFac, on=('SEGID','SOURCE'), how="left")

dfMdlVolWithFac.fillna(0, inplace=True)

display(dfMdlVolWithFac)

,SEGID,CO_FIPS,SUBAREAID,SOURCE,YEAR,DY_VOL,FT,ATYPENAME,LANES,inTdm,FAC_FAL,FAC_SPR,FAC_WDAVG,FAC_SPRFAL
0,0006_000.0,27.0,0.0,Segments_State_20231221_Draft.shp,2019,126.5,2.0,Rural,1.0,1.0,1.0316,1.0276,0.9840,1.02960
1,0006_000.0,27.0,0.0,Segments_State_20231221_Draft.shp,2023,99.7,2.0,Rural,1.0,1.0,1.0316,1.0276,0.9840,1.02960
2,0006_000.0,27.0,0.0,Segments_State_20231221_Draft.shp,2028,101.7,2.0,Rural,1.0,1.0,1.0316,1.0276,0.9840,1.02960
3,0006_000.0,27.0,0.0,Segments_State_20231221_Draft.shp,2032,103.8,2.0,Rural,1.0,1.0,1.0316,1.0276,0.9840,1.02960
4,0006_000.0,27.0,0.0,Segments_State_20231221_Draft.shp,2042,108.1,2.0,Rural,1.0,1.0,1.0316,1.0276,0.9840,1.02960
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57367,WFRC_8430,35.0,1.0,WF_Segments_20240326_Draft.shp,2023,6392.6,3.0,Suburban,2.0,1.0,1.0243,1.0104,1.0924,1.01735
57368,WFRC_8430,35.0,1.0,WF_Segments_20240326_Draft.shp,2028,7340.4,3.0,Suburban,2.0,1.0,1.0243,1.0104,1.0924,1.01735
57369,WFRC_8430,35.0,1.0,WF_Segments_20240326_Draft.shp,2032,6968.0,3.0,Suburban,2.0,1.0,1.0243,1.0104,1.0924,1.01735
57370,WFRC_8430,35.0,1.0,WF_Segments_20240326_Draft.shp,2042,8527.4,3.0,Suburban,2.0,1.0,1.0243,1.0104,1.0924,1.01735


In [10]:
dfMdlVolWithFac[dfMdlVolWithFac['SEGID']=='1695_000.0']

,SEGID,CO_FIPS,SUBAREAID,SOURCE,YEAR,DY_VOL,FT,ATYPENAME,LANES,inTdm,FAC_FAL,FAC_SPR,FAC_WDAVG,FAC_SPRFAL
10098,1695_000.0,19.0,0.0,Segments_State_20231221_Draft.shp,2019,1558.0,4.0,Transition,1.0,1.0,1.0827,1.2456,0.9243,1.16415
10099,1695_000.0,19.0,0.0,Segments_State_20231221_Draft.shp,2023,1515.1,4.0,Transition,1.0,1.0,1.0827,1.2456,0.9243,1.16415
10100,1695_000.0,19.0,0.0,Segments_State_20231221_Draft.shp,2028,1673.5,4.0,Transition,1.0,1.0,1.0827,1.2456,0.9243,1.16415
10101,1695_000.0,19.0,0.0,Segments_State_20231221_Draft.shp,2032,1787.5,4.0,Transition,1.0,1.0,1.0827,1.2456,0.9243,1.16415
10102,1695_000.0,19.0,0.0,Segments_State_20231221_Draft.shp,2042,1959.2,4.0,Transition,1.0,1.0,1.0827,1.2456,0.9243,1.16415
10103,1695_000.0,19.0,0.0,Segments_State_20231221_Draft.shp,2050,1991.5,4.0,Transition,1.0,1.0,1.0827,1.2456,0.9243,1.16415


## get and use avg where factors zero... no match

In [11]:
dfFindZeroFac = dfMdlVolWithFac.copy()
dfFindZeroFac.loc[(dfFindZeroFac['FAC_WDAVG']==0), 'checkZero'] = 1
dfFindZeroFac.groupby(['CO_FIPS','YEAR']).agg(numSegs=('SEGID','count'),numSegsWithFacZero=('checkZero','sum'))

numSegs  numSegsWithFacZero
CO_FIPS YEAR                             
1.0     2019       96                 0.0
        2023       96                 0.0
        2028       96                 0.0
        2032       96                 0.0
        2042       96                 0.0
...               ...                 ...
57.0    2023      608                 0.0
        2028      608                 0.0
        2032      608                 0.0
        2042      608                 0.0
        2050      608                 0.0

[174 rows x 2 columns]

In [12]:
dfNonZeroFac = dfFindZeroFac[~(dfFindZeroFac['checkZero']== 1)].drop(columns=['checkZero'])
display(dfNonZeroFac)
dfZeroFac    = dfFindZeroFac[ (dfFindZeroFac['checkZero']== 1)].drop(columns=['checkZero'])
display(dfZeroFac)

,SEGID,CO_FIPS,SUBAREAID,SOURCE,YEAR,DY_VOL,FT,ATYPENAME,LANES,inTdm,FAC_FAL,FAC_SPR,FAC_WDAVG,FAC_SPRFAL
0,0006_000.0,27.0,0.0,Segments_State_20231221_Draft.shp,2019,126.5,2.0,Rural,1.0,1.0,1.0316,1.0276,0.9840,1.02960
1,0006_000.0,27.0,0.0,Segments_State_20231221_Draft.shp,2023,99.7,2.0,Rural,1.0,1.0,1.0316,1.0276,0.9840,1.02960
2,0006_000.0,27.0,0.0,Segments_State_20231221_Draft.shp,2028,101.7,2.0,Rural,1.0,1.0,1.0316,1.0276,0.9840,1.02960
3,0006_000.0,27.0,0.0,Segments_State_20231221_Draft.shp,2032,103.8,2.0,Rural,1.0,1.0,1.0316,1.0276,0.9840,1.02960
4,0006_000.0,27.0,0.0,Segments_State_20231221_Draft.shp,2042,108.1,2.0,Rural,1.0,1.0,1.0316,1.0276,0.9840,1.02960
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57367,WFRC_8430,35.0,1.0,WF_Segments_20240326_Draft.shp,2023,6392.6,3.0,Suburban,2.0,1.0,1.0243,1.0104,1.0924,1.01735
57368,WFRC_8430,35.0,1.0,WF_Segments_20240326_Draft.shp,2028,7340.4,3.0,Suburban,2.0,1.0,1.0243,1.0104,1.0924,1.01735
57369,WFRC_8430,35.0,1.0,WF_Segments_20240326_Draft.shp,2032,6968.0,3.0,Suburban,2.0,1.0,1.0243,1.0104,1.0924,1.01735
57370,WFRC_8430,35.0,1.0,WF_Segments_20240326_Draft.shp,2042,8527.4,3.0,Suburban,2.0,1.0,1.0243,1.0104,1.0924,1.01735


,SEGID,CO_FIPS,SUBAREAID,SOURCE,YEAR,DY_VOL,FT,ATYPENAME,LANES,inTdm,FAC_FAL,FAC_SPR,FAC_WDAVG,FAC_SPRFAL
57354,WFRC_8474,35.0,1.0,WF_Segments_20240326_Draft.shp,2019,1.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
57355,WFRC_8474,35.0,1.0,WF_Segments_20240326_Draft.shp,2023,1.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
57356,WFRC_8474,35.0,1.0,WF_Segments_20240326_Draft.shp,2028,1.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
57357,WFRC_8474,35.0,1.0,WF_Segments_20240326_Draft.shp,2032,1.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
57358,WFRC_8474,35.0,1.0,WF_Segments_20240326_Draft.shp,2042,1.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
57359,WFRC_8474,35.0,1.0,WF_Segments_20240326_Draft.shp,2050,1.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
57360,WFRC_8475,35.0,1.0,WF_Segments_20240326_Draft.shp,2019,1.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
57361,WFRC_8475,35.0,1.0,WF_Segments_20240326_Draft.shp,2023,1.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
57362,WFRC_8475,35.0,1.0,WF_Segments_20240326_Draft.shp,2028,1.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
57363,WFRC_8475,35.0,1.0,WF_Segments_20240326_Draft.shp,2032,1.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
dfZeroFacSaAtCombos = dfZeroFac[['CO_FIPS','ATYPENAME']].drop_duplicates()
display(dfZeroFacSaAtCombos)

,CO_FIPS,ATYPENAME
57354,35.0,0


In [14]:
dfMdlVolWithFac_Filtered = pd.merge(dfMdlVolWithFac, dfZeroFacSaAtCombos, on=['CO_FIPS', 'ATYPENAME'], how='inner')
dfMdlVolWithFac_Filtered


,SEGID,CO_FIPS,SUBAREAID,SOURCE,YEAR,DY_VOL,FT,ATYPENAME,LANES,inTdm,FAC_FAL,FAC_SPR,FAC_WDAVG,FAC_SPRFAL
0,0131_000.0,35.0,1.0,WF_Segments_20240326_Draft.shp,2019,1.0,0.0,0,0.0,0.0,1.0324,1.0107,1.0918,1.02155
1,0190_015.7,35.0,1.0,WF_Segments_20240326_Draft.shp,2019,1.0,0.0,0,0.0,0.0,0.8387,0.7791,0.7907,0.80890
2,0190_015.7,35.0,1.0,WF_Segments_20240326_Draft.shp,2023,1.0,0.0,0,0.0,0.0,0.8387,0.7791,0.7907,0.80890
3,0190_015.7,35.0,1.0,WF_Segments_20240326_Draft.shp,2028,1.0,0.0,0,0.0,0.0,0.8387,0.7791,0.7907,0.80890
4,0190_015.7,35.0,1.0,WF_Segments_20240326_Draft.shp,2032,1.0,0.0,0,0.0,0.0,0.8387,0.7791,0.7907,0.80890
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1272,WFRC_8475,35.0,1.0,WF_Segments_20240326_Draft.shp,2023,1.0,0.0,0,0.0,0.0,0.0000,0.0000,0.0000,0.00000
1273,WFRC_8475,35.0,1.0,WF_Segments_20240326_Draft.shp,2028,1.0,0.0,0,0.0,0.0,0.0000,0.0000,0.0000,0.00000
1274,WFRC_8475,35.0,1.0,WF_Segments_20240326_Draft.shp,2032,1.0,0.0,0,0.0,0.0,0.0000,0.0000,0.0000,0.00000
1275,WFRC_8475,35.0,1.0,WF_Segments_20240326_Draft.shp,2042,1.0,0.0,0,0.0,0.0,0.0000,0.0000,0.0000,0.00000


In [15]:
dfMdlVolWithFac_Filtered_avg = dfMdlVolWithFac_Filtered.groupby(['CO_FIPS','ATYPENAME'],as_index=False).agg(FAC_FAL = ('FAC_FAL','mean'),FAC_SPR = ('FAC_SPR','mean'),FAC_WDAVG = ('FAC_WDAVG','mean'),FAC_SPRFAL = ('FAC_SPRFAL','mean'))
dfMdlVolWithFac_Filtered_avg

,CO_FIPS,ATYPENAME,FAC_FAL,FAC_SPR,FAC_WDAVG,FAC_SPRFAL
0,35.0,0,1.01381,1.001589,1.062112,1.0077


In [16]:
dfZeroFacWithAvg = pd.merge(dfZeroFac.drop(columns=['FAC_FAL','FAC_SPR','FAC_WDAVG','FAC_SPRFAL']),dfMdlVolWithFac_Filtered_avg, on=('CO_FIPS','ATYPENAME'),how='left')
dfZeroFacWithAvg

,SEGID,CO_FIPS,SUBAREAID,SOURCE,YEAR,DY_VOL,FT,ATYPENAME,LANES,inTdm,FAC_FAL,FAC_SPR,FAC_WDAVG,FAC_SPRFAL
0,WFRC_8474,35.0,1.0,WF_Segments_20240326_Draft.shp,2019,1.0,0.0,0,0.0,0.0,1.01381,1.001589,1.062112,1.0077
1,WFRC_8474,35.0,1.0,WF_Segments_20240326_Draft.shp,2023,1.0,0.0,0,0.0,0.0,1.01381,1.001589,1.062112,1.0077
2,WFRC_8474,35.0,1.0,WF_Segments_20240326_Draft.shp,2028,1.0,0.0,0,0.0,0.0,1.01381,1.001589,1.062112,1.0077
3,WFRC_8474,35.0,1.0,WF_Segments_20240326_Draft.shp,2032,1.0,0.0,0,0.0,0.0,1.01381,1.001589,1.062112,1.0077
4,WFRC_8474,35.0,1.0,WF_Segments_20240326_Draft.shp,2042,1.0,0.0,0,0.0,0.0,1.01381,1.001589,1.062112,1.0077
5,WFRC_8474,35.0,1.0,WF_Segments_20240326_Draft.shp,2050,1.0,0.0,0,0.0,0.0,1.01381,1.001589,1.062112,1.0077
6,WFRC_8475,35.0,1.0,WF_Segments_20240326_Draft.shp,2019,1.0,0.0,0,0.0,0.0,1.01381,1.001589,1.062112,1.0077
7,WFRC_8475,35.0,1.0,WF_Segments_20240326_Draft.shp,2023,1.0,0.0,0,0.0,0.0,1.01381,1.001589,1.062112,1.0077
8,WFRC_8475,35.0,1.0,WF_Segments_20240326_Draft.shp,2028,1.0,0.0,0,0.0,0.0,1.01381,1.001589,1.062112,1.0077
9,WFRC_8475,35.0,1.0,WF_Segments_20240326_Draft.shp,2032,1.0,0.0,0,0.0,0.0,1.01381,1.001589,1.062112,1.0077


In [17]:
dfMdlVolWithFac = pd.concat([dfZeroFacWithAvg,dfNonZeroFac], ignore_index=True)
dfMdlVolWithFac

,SEGID,CO_FIPS,SUBAREAID,SOURCE,YEAR,DY_VOL,FT,ATYPENAME,LANES,inTdm,FAC_FAL,FAC_SPR,FAC_WDAVG,FAC_SPRFAL
0,WFRC_8474,35.0,1.0,WF_Segments_20240326_Draft.shp,2019,1.0,0.0,0,0.0,0.0,1.01381,1.001589,1.062112,1.00770
1,WFRC_8474,35.0,1.0,WF_Segments_20240326_Draft.shp,2023,1.0,0.0,0,0.0,0.0,1.01381,1.001589,1.062112,1.00770
2,WFRC_8474,35.0,1.0,WF_Segments_20240326_Draft.shp,2028,1.0,0.0,0,0.0,0.0,1.01381,1.001589,1.062112,1.00770
3,WFRC_8474,35.0,1.0,WF_Segments_20240326_Draft.shp,2032,1.0,0.0,0,0.0,0.0,1.01381,1.001589,1.062112,1.00770
4,WFRC_8474,35.0,1.0,WF_Segments_20240326_Draft.shp,2042,1.0,0.0,0,0.0,0.0,1.01381,1.001589,1.062112,1.00770
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57367,WFRC_8430,35.0,1.0,WF_Segments_20240326_Draft.shp,2023,6392.6,3.0,Suburban,2.0,1.0,1.02430,1.010400,1.092400,1.01735
57368,WFRC_8430,35.0,1.0,WF_Segments_20240326_Draft.shp,2028,7340.4,3.0,Suburban,2.0,1.0,1.02430,1.010400,1.092400,1.01735
57369,WFRC_8430,35.0,1.0,WF_Segments_20240326_Draft.shp,2032,6968.0,3.0,Suburban,2.0,1.0,1.02430,1.010400,1.092400,1.01735
57370,WFRC_8430,35.0,1.0,WF_Segments_20240326_Draft.shp,2042,8527.4,3.0,Suburban,2.0,1.0,1.02430,1.010400,1.092400,1.01735


# drop where overlapping forecasts (same segids in different subareas)

In [18]:
# get segids only
dfSegIds = dfMdlVolWithFac[['SEGID']].drop_duplicates()
display(dfSegIds)

# filter by 
display(dfMdlVolWithFac[['SEGID','SUBAREAID']].drop_duplicates())

,SEGID
0,WFRC_8474
6,WFRC_8475
12,0006_000.0
18,0006_000.7
24,0006_016.0
...,...
57342,WFRC_8470
57348,WFRC_8471
57354,WFRC_8472
57360,WFRC_8473


,SEGID,SUBAREAID
0,WFRC_8474,1.0
6,WFRC_8475,1.0
12,0006_000.0,0.0
18,0006_000.7,0.0
24,0006_016.0,0.0
...,...,...
57342,WFRC_8470,1.0
57348,WFRC_8471,1.0
57354,WFRC_8472,1.0
57360,WFRC_8473,1.0


In [19]:
# Assuming dfMdlVolWithFac is your initial DataFrame
df_filtered = dfMdlVolWithFac[['SEGID','SUBAREAID']].drop_duplicates()

#df_filtered.rename(columns={'SUBAREAID':'TDM_SUBAREAID','SAID_FAC':'SEG_SUBAREAID'}, inplace=True)

# Find duplicates based on 'SEGID'
duplicates = df_filtered.duplicated(subset=['SEGID'], keep=False)

# Filter the DataFrame to include only the duplicates
df_duplicates = df_filtered[duplicates]

df_duplicates = df_duplicates.sort_values(by=['SEGID','SUBAREAID'])

# This will give you a DataFrame with only the rows where 'SEGID' is duplicated
df_duplicates

,SEGID,SUBAREAID
44148,2863_000.0,1.0
25236,2863_000.0,4.0


In [20]:
## Assuming dfMdlVolWithFac is your initial DataFrame
#df_filtered = dfMdlVolWithFac[['SEGID','SUBAREAID','SAID_FAC']].drop_duplicates()
#
#df_filtered.rename(columns={'SUBAREAID':'TDM_SUBAREAID','SAID_FAC':'SEG_SUBAREAID'}, inplace=True)
#
## Find duplicates based on 'SEGID'
#duplicates = df_filtered.duplicated(subset=['SEGID'], keep=False)
#
## Filter the DataFrame to include only the duplicates
#df_duplicates = df_filtered[duplicates]
#
#df_duplicates = df_duplicates.sort_values(by=['SEGID','TDM_SUBAREAID'])
#
#df_duplicates = df_duplicates.set_index(['SEGID'])
#
#df_duplicates = df_duplicates.astype(int)
#
## This will give you a DataFrame with only the rows where 'SEGID' is duplicated
#df_duplicates[df_duplicates['TDM_SUBAREAID']!=df_duplicates['SEG_SUBAREAID']]

  - 0015_042.2: Remove SEGID from links in Dixie TDM (3)
 - 0065_002.4: Change SUBAREAID on links in USTM from 0 to 1
 - 0150_029.4, 015_030.0, 015_030.3 both segments where route leaves Summit County briefly into the corner of Duchesne County, so segments are not geographically located in SuWs assume should be in SuWs Model... so, seg summary should be updated... probably have to custom write exclusion into code, since SUBAREAID was calculated with spatial join
 - 1822_000.0: Majority of segment is in Utah County... Should split segment at county line, or change SUBAREAID on links in USTM from 0 to 1
 - 2865_019.4: Majority of segment is in Utah County... Should split segment at county line, or change SUBAREAID on links in USTM from 0 to 1


In [21]:
duplicateSegsInTdms

,SEGID,removeSUBAREAID
0,0015_042.2,3
1,0065_002.4,0
2,0150_029.4,0
3,0150_030.0,0
4,0150_033.3,0
5,1822_000.0,0
6,2863_000.0,4
7,2865_019.4,4


In [22]:
# Example dataframes
# df1 = pd.DataFrame({'SEGID': [1, 2], 'removeSUBAREAID': [101, 102]})
# dfMdlVolWithFac = pd.DataFrame({'SEGID': [1, 2, 3], 'SUBAREAID': [101, 102, 103], 'OtherColumn': ['A', 'B', 'C']})

# Assuming 'removeSUBAREAID' in df1 matches 'SUBAREAID' in dfMdlVolWithFac, and you want to remove records from dfMdlVolWithFac that have matching 'SEGID' and 'SUBAREAID'
# If 'removeSUBAREAID' is not meant to directly match 'SUBAREAID', adjust the merge accordingly.

# Merge the dataframes on 'SEGID' and 'SUBAREAID'/'removeSUBAREAID', indicating records to remove
merged_df = pd.merge(dfMdlVolWithFac, duplicateSegsInTdms, left_on=['SEGID', 'SUBAREAID'], right_on=['SEGID', 'removeSUBAREAID'], how='left', indicator=True)

# Filter out the records that have a match
filtered_df = merged_df[merged_df['_merge'] == 'left_only']

# Drop the columns added from df1 and the merge indicator to get back to the original dfMdlVolWithFac format
final_df = filtered_df.drop(columns=['removeSUBAREAID', '_merge'])

# final_df will be your dfMdlVolWithFac dataframe with the matched records removed
dfMdlVolWithFacDupRem = final_df
dfMdlVolWithFacDupRem

,SEGID,CO_FIPS,SUBAREAID,SOURCE,YEAR,DY_VOL,FT,ATYPENAME,LANES,inTdm,FAC_FAL,FAC_SPR,FAC_WDAVG,FAC_SPRFAL
0,WFRC_8474,35.0,1.0,WF_Segments_20240326_Draft.shp,2019,1.0,0.0,0,0.0,0.0,1.01381,1.001589,1.062112,1.00770
1,WFRC_8474,35.0,1.0,WF_Segments_20240326_Draft.shp,2023,1.0,0.0,0,0.0,0.0,1.01381,1.001589,1.062112,1.00770
2,WFRC_8474,35.0,1.0,WF_Segments_20240326_Draft.shp,2028,1.0,0.0,0,0.0,0.0,1.01381,1.001589,1.062112,1.00770
3,WFRC_8474,35.0,1.0,WF_Segments_20240326_Draft.shp,2032,1.0,0.0,0,0.0,0.0,1.01381,1.001589,1.062112,1.00770
4,WFRC_8474,35.0,1.0,WF_Segments_20240326_Draft.shp,2042,1.0,0.0,0,0.0,0.0,1.01381,1.001589,1.062112,1.00770
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57367,WFRC_8430,35.0,1.0,WF_Segments_20240326_Draft.shp,2023,6392.6,3.0,Suburban,2.0,1.0,1.02430,1.010400,1.092400,1.01735
57368,WFRC_8430,35.0,1.0,WF_Segments_20240326_Draft.shp,2028,7340.4,3.0,Suburban,2.0,1.0,1.02430,1.010400,1.092400,1.01735
57369,WFRC_8430,35.0,1.0,WF_Segments_20240326_Draft.shp,2032,6968.0,3.0,Suburban,2.0,1.0,1.02430,1.010400,1.092400,1.01735
57370,WFRC_8430,35.0,1.0,WF_Segments_20240326_Draft.shp,2042,8527.4,3.0,Suburban,2.0,1.0,1.02430,1.010400,1.092400,1.01735


In [23]:
# CHECK TO SEE IF DUPLICATES STILL REMAIN

# Assuming dfMdlVol is your initial DataFrame
df_filtered2 = dfMdlVolWithFacDupRem[['SEGID','SUBAREAID']].drop_duplicates()

# Find duplicates based on 'SEGID'
duplicates2 = df_filtered2.duplicated(subset=['SEGID'], keep=False)

# Filter the DataFrame to include only the duplicates2
df_duplicates2 = df_filtered2[duplicates2]

# This will give you a DataFrame with only the rows where 'SEGID' is duplicated
df_duplicates2

,SEGID,SUBAREAID


# Calculate Model AADT

In [24]:
import numpy as np

# Your existing code for merging and filling NA
_df = dfMdlVolWithFacDupRem.copy()

# calculate AADT by subareaid
# divide by colWeekdayFactor
_df.loc[(~(_df['SUBAREAID'].isin(subareaidsThatUseSprFalFac)) & (_df[colFacWdAvg]> 0)), 'modAadt'] = (_df['DY_VOL'] / _df[colFacWdAvg])
_df.loc[(~(_df['SUBAREAID'].isin(subareaidsThatUseSprFalFac)) & (_df[colFacWdAvg]==0)), 'modAadt'] = 0

# divide by average of Spring and Fall only for Summit/Wasatch Model
_df.loc[(_df['SUBAREAID'].isin(subareaidsThatUseSprFalFac) & (_df['FAC_SPRFAL']>0 )), 'modAadt'] = (_df['DY_VOL'] / _df[colFacWdAvg] / _df['FAC_SPRFAL'])
_df.loc[(_df['SUBAREAID'].isin(subareaidsThatUseSprFalFac) & (_df['FAC_SPRFAL']==0)), 'modAadt'] = 0

# Convert to integer as you had before
_df['modAadt'] = _df['modAadt'].round(0).astype(int)

dfModAadt = _df

# Display the DataFrame
display(dfModAadt)

,SEGID,CO_FIPS,SUBAREAID,SOURCE,YEAR,DY_VOL,FT,ATYPENAME,LANES,inTdm,FAC_FAL,FAC_SPR,FAC_WDAVG,FAC_SPRFAL,modAadt
0,WFRC_8474,35.0,1.0,WF_Segments_20240326_Draft.shp,2019,1.0,0.0,0,0.0,0.0,1.01381,1.001589,1.062112,1.00770,1
1,WFRC_8474,35.0,1.0,WF_Segments_20240326_Draft.shp,2023,1.0,0.0,0,0.0,0.0,1.01381,1.001589,1.062112,1.00770,1
2,WFRC_8474,35.0,1.0,WF_Segments_20240326_Draft.shp,2028,1.0,0.0,0,0.0,0.0,1.01381,1.001589,1.062112,1.00770,1
3,WFRC_8474,35.0,1.0,WF_Segments_20240326_Draft.shp,2032,1.0,0.0,0,0.0,0.0,1.01381,1.001589,1.062112,1.00770,1
4,WFRC_8474,35.0,1.0,WF_Segments_20240326_Draft.shp,2042,1.0,0.0,0,0.0,0.0,1.01381,1.001589,1.062112,1.00770,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57367,WFRC_8430,35.0,1.0,WF_Segments_20240326_Draft.shp,2023,6392.6,3.0,Suburban,2.0,1.0,1.02430,1.010400,1.092400,1.01735,5852
57368,WFRC_8430,35.0,1.0,WF_Segments_20240326_Draft.shp,2028,7340.4,3.0,Suburban,2.0,1.0,1.02430,1.010400,1.092400,1.01735,6720
57369,WFRC_8430,35.0,1.0,WF_Segments_20240326_Draft.shp,2032,6968.0,3.0,Suburban,2.0,1.0,1.02430,1.010400,1.092400,1.01735,6379
57370,WFRC_8430,35.0,1.0,WF_Segments_20240326_Draft.shp,2042,8527.4,3.0,Suburban,2.0,1.0,1.02430,1.010400,1.092400,1.01735,7806


# Get observed base year volumes

In [25]:
# GET BASE YEAR OBSERVED

# read intermediate CSV with base year observed AADT
_dfAadt = pd.read_csv('intermediate/aadt.csv')

# get SEGID and SUBAREAIDs from AADT dataframe  and merge to the desired source as indicated in dfObsVolSource
_dfSource = pd.merge(dfModAadt[['SEGID','SUBAREAID']].drop_duplicates(),dfSegsForFacSource,on=('SEGID','SUBAREAID'),how='left')

_df = pd.merge(_dfSource, _dfAadt, on=('SEGID','SOURCE'))

# filter by source and year
_df = _dfAadt[(_dfAadt['YEAR']<=baseYear)]

# get max row of max year for each segid... incase there is no observed 2019 data, but maybe a earlier year
# Then, group by 'SEGID' and find the index of the maximum 'YEAR' for each group
idx = _df.groupby('SEGID')['YEAR'].idxmax()

# Use the index to filter the rows
_df = _df.loc[idx]

_df = _df[['SEGID','AADT']]
_df.rename(columns={'AADT':'obsAadt'}, inplace=True)

dfObsAadt_BaseYear = _df

display(dfObsAadt_BaseYear)

,SEGID,obsAadt
231665,0006_000.0,415
231708,0006_000.7,415
231751,0006_016.0,415
231794,0006_046.0,372
231837,0006_060.2,372
...,...,...
133310,WFRC_8220,17096
133334,WFRC_8262,6672
133361,WFRC_8263,6672
133400,WFRC_8419,18324


In [26]:
dfObsAadt_BaseYear[dfObsAadt_BaseYear['SEGID']=='1695_000.0']

,SEGID,obsAadt
303597,1695_000.0,4122


In [27]:
# CALCULATE ADJUSTMENT FACTOR FROM BASE YEAR MODEL VS OBSERVED
dfModAadt_BaseYear = dfModAadt[dfModAadt['YEAR']==2019]

_df = pd.DataFrame.merge(dfModAadt_BaseYear[use_fields + ['modAadt']], dfObsAadt_BaseYear[['SEGID','obsAadt']], on='SEGID', how='outer')
_df.fillna(0,inplace=True)

_df['aadtAdjFac'] = _df['obsAadt'] - _df['modAadt']

_df.loc[(_df['obsAadt']==0), 'aadtAdjFac'] = 0

display(_df[_df['modAadt']==0])

dfModAdjFactor = _df[['SEGID','aadtAdjFac']]

display(dfModAdjFactor)


,SEGID,SUBAREAID,DY_VOL,FT,ATYPENAME,LANES,modAadt,obsAadt,aadtAdjFac
183,0019_004.5,0.0,0.0,4.0,Rural,1.0,0.0,2749.0,2749.0
251,0025_000.0,0.0,0.0,4.0,Rural,1.0,0.0,179.0,179.0
252,0025_002.9,0.0,0.0,4.0,Rural,1.0,0.0,179.0,179.0
914,0122_000.0,0.0,0.0,4.0,Rural,1.0,0.0,77.0,77.0
1198,0275_000.0,0.0,0.0,4.0,Rural,1.0,0.0,87.0,87.0
...,...,...,...,...,...,...,...,...,...
9584,2964_000.4,0.0,0.0,0.0,0,0.0,0.0,5257.0,5257.0
9585,2996_000.33,0.0,0.0,0.0,0,0.0,0.0,12427.0,12427.0
9586,3006_000.3,0.0,0.0,0.0,0,0.0,0.0,9411.0,9411.0
9587,3020_001.43,0.0,0.0,0.0,0,0.0,0.0,25161.0,25161.0


,SEGID,aadtAdjFac
0,WFRC_8474,0.0
1,WFRC_8475,0.0
2,0006_000.0,286.0
3,0006_000.7,195.0
4,0006_016.0,334.0
...,...,...
9584,2964_000.4,5257.0
9585,2996_000.33,12427.0
9586,3006_000.3,9411.0
9587,3020_001.43,25161.0


In [28]:
# Function to round based on the given ranges
def custom_rounding(value):
    if 0 <= value < 100:
        return round(value / 10) * 10
    elif 100 <= value < 1000:
        return round(value / 50) * 50
    elif 1000 <= value < 10000:
        return round(value / 100) * 100
    elif 10000 <= value < 100000:
        return round(value / 500) * 500
    elif value >= 100000:
        return round(value / 1000) * 1000
    else:
        return value

In [29]:
# ADJUST ALL MODEL VOLUMES BASED ON BASE YEAR ADJ FACTOR

dfModWithAdj = pd.DataFrame.merge(dfModAadt, dfModAdjFactor, on='SEGID', how='left')
dfModWithAdj.fillna(0,inplace=True)
dfModWithAdj['aadtAdjFac'] = dfModWithAdj['aadtAdjFac'].astype(int)
dfModWithAdj['modForecast'] = (dfModWithAdj['modAadt'] + dfModWithAdj['aadtAdjFac']).apply(custom_rounding).astype(int)
dfModWithAdj

# remove 2019 since it is base
#dfModWithAdjNo2019 = dfModWithAdj[dfModWithAdj['YEAR']>2019]

# sort
#dfModWithAdjNo2019 = dfModWithAdjNo2019.sort_values(by=['SEGID', 'YEAR'], ascending=[True, True])

#dfModWithAdjNo2019

,SEGID,CO_FIPS,SUBAREAID,SOURCE,YEAR,DY_VOL,FT,ATYPENAME,LANES,inTdm,FAC_FAL,FAC_SPR,FAC_WDAVG,FAC_SPRFAL,modAadt,aadtAdjFac,modForecast
0,WFRC_8474,35.0,1.0,WF_Segments_20240326_Draft.shp,2019,1.0,0.0,0,0.0,0.0,1.01381,1.001589,1.062112,1.00770,1,0,0
1,WFRC_8474,35.0,1.0,WF_Segments_20240326_Draft.shp,2023,1.0,0.0,0,0.0,0.0,1.01381,1.001589,1.062112,1.00770,1,0,0
2,WFRC_8474,35.0,1.0,WF_Segments_20240326_Draft.shp,2028,1.0,0.0,0,0.0,0.0,1.01381,1.001589,1.062112,1.00770,1,0,0
3,WFRC_8474,35.0,1.0,WF_Segments_20240326_Draft.shp,2032,1.0,0.0,0,0.0,0.0,1.01381,1.001589,1.062112,1.00770,1,0,0
4,WFRC_8474,35.0,1.0,WF_Segments_20240326_Draft.shp,2042,1.0,0.0,0,0.0,0.0,1.01381,1.001589,1.062112,1.00770,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57343,WFRC_8430,35.0,1.0,WF_Segments_20240326_Draft.shp,2023,6392.6,3.0,Suburban,2.0,1.0,1.02430,1.010400,1.092400,1.01735,5852,0,5900
57344,WFRC_8430,35.0,1.0,WF_Segments_20240326_Draft.shp,2028,7340.4,3.0,Suburban,2.0,1.0,1.02430,1.010400,1.092400,1.01735,6720,0,6700
57345,WFRC_8430,35.0,1.0,WF_Segments_20240326_Draft.shp,2032,6968.0,3.0,Suburban,2.0,1.0,1.02430,1.010400,1.092400,1.01735,6379,0,6400
57346,WFRC_8430,35.0,1.0,WF_Segments_20240326_Draft.shp,2042,8527.4,3.0,Suburban,2.0,1.0,1.02430,1.010400,1.092400,1.01735,7806,0,7800


In [30]:
dfModWithAdj[dfModWithAdj['SEGID']=='1695_000.0']

,SEGID,CO_FIPS,SUBAREAID,SOURCE,YEAR,DY_VOL,FT,ATYPENAME,LANES,inTdm,FAC_FAL,FAC_SPR,FAC_WDAVG,FAC_SPRFAL,modAadt,aadtAdjFac,modForecast
10092,1695_000.0,19.0,0.0,Segments_State_20231221_Draft.shp,2019,1558.0,4.0,Transition,1.0,1.0,1.0827,1.2456,0.9243,1.16415,1686,2436,4100
10093,1695_000.0,19.0,0.0,Segments_State_20231221_Draft.shp,2023,1515.1,4.0,Transition,1.0,1.0,1.0827,1.2456,0.9243,1.16415,1639,2436,4100
10094,1695_000.0,19.0,0.0,Segments_State_20231221_Draft.shp,2028,1673.5,4.0,Transition,1.0,1.0,1.0827,1.2456,0.9243,1.16415,1811,2436,4200
10095,1695_000.0,19.0,0.0,Segments_State_20231221_Draft.shp,2032,1787.5,4.0,Transition,1.0,1.0,1.0827,1.2456,0.9243,1.16415,1934,2436,4400
10096,1695_000.0,19.0,0.0,Segments_State_20231221_Draft.shp,2042,1959.2,4.0,Transition,1.0,1.0,1.0827,1.2456,0.9243,1.16415,2120,2436,4600
10097,1695_000.0,19.0,0.0,Segments_State_20231221_Draft.shp,2050,1991.5,4.0,Transition,1.0,1.0,1.0827,1.2456,0.9243,1.16415,2155,2436,4600


In [31]:
# check if base year adjusted equals base year observed

_df1 = dfModWithAdj[dfModWithAdj['YEAR']==baseYear]
_df2 = dfObsAadt_BaseYear

_df3 = pd.DataFrame.merge(_df1, _df2, on='SEGID')

_df3['ModVsObs'] = _df3['modForecast'] - _df3['obsAadt']

display(_df3[_df3['ModVsObs']!=0])


,SEGID,CO_FIPS,SUBAREAID,SOURCE,YEAR,DY_VOL,FT,ATYPENAME,LANES,inTdm,FAC_FAL,FAC_SPR,FAC_WDAVG,FAC_SPRFAL,modAadt,aadtAdjFac,modForecast,obsAadt,ModVsObs
0,0006_000.0,27.0,0.0,Segments_State_20231221_Draft.shp,2019,126.5,2.0,Rural,1.0,1.0,1.0316,1.0276,0.9840,1.02960,129,286,400,415,-15
1,0006_000.7,27.0,0.0,Segments_State_20231221_Draft.shp,2019,216.6,2.0,Rural,1.0,1.0,1.0316,1.0276,0.9840,1.02960,220,195,400,415,-15
2,0006_016.0,27.0,0.0,Segments_State_20231221_Draft.shp,2019,79.9,2.0,Rural,1.0,1.0,1.0316,1.0276,0.9840,1.02960,81,334,400,415,-15
3,0006_046.0,27.0,0.0,Segments_State_20231221_Draft.shp,2019,79.9,2.0,Rural,1.0,1.0,1.0316,1.0276,0.9840,1.02960,81,291,350,372,-22
4,0006_060.2,27.0,0.0,Segments_State_20231221_Draft.shp,2019,83.6,2.0,Rural,1.0,1.0,1.0316,1.0276,0.9840,1.02960,85,287,350,372,-22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7280,WFRC_8220,35.0,1.0,WF_Segments_20240326_Draft.shp,2019,12301.9,3.0,Suburban,3.0,1.0,1.0243,1.0104,1.0924,1.01735,11261,5835,17000,17096,-96
7281,WFRC_8262,35.0,1.0,WF_Segments_20240326_Draft.shp,2019,1.0,0.0,0,0.0,0.0,1.0243,1.0104,1.0924,1.01735,1,6671,6700,6672,28
7282,WFRC_8263,35.0,1.0,WF_Segments_20240326_Draft.shp,2019,4659.0,4.0,Suburban,2.0,1.0,1.0243,1.0104,1.0924,1.01735,4265,2407,6700,6672,28
7283,WFRC_8419,11.0,1.0,WF_Segments_20240326_Draft.shp,2019,44116.6,3.0,Urban,4.0,1.0,1.0095,1.0117,1.0946,1.01060,40304,-21980,18500,18324,176


In [32]:
lstLockedSegments = gdfSegments[gdfSegments['F_AREA'].isin(lstLockForecastArea)]['SEGID'].drop_duplicates().to_list()
display(lstLockedSegments)

# merge locked with unlocked

if len(lstLockedSegments):

    print('Combine locked and unlocked')

    # export final file
    dfModWithAdj_Prev = pd.read_csv('intermediate/model-forecasts.csv')

    dfModWithAdj_Locked = dfModWithAdj_Prev[dfModWithAdj_Prev['SEGID'].isin(lstLockedSegments)].copy()

    dfModWithAdj_Unlocked = dfModWithAdj[~dfModWithAdj['SEGID'].isin(lstLockedSegments)].copy()

    # Concatenate the locked and unlocked DataFrames
    dfModWithAdjMerged = pd.concat([dfModWithAdj_Locked, dfModWithAdj_Unlocked])

    # Reset the index of the final DataFrame, if needed
    dfModWithAdjMerged.reset_index(drop=True, inplace=True)

else:
    dfModWithAdjMerged = dfModWithAdj

dfModWithAdjMerged

['0006_141.0',
 '0006_146.9',
 '0006_149.9',
 '0006_150.6',
 '0006_152.6',
 '0006_152.9',
 '0006_155.8',
 '0006_155.9',
 '0006_157.3',
 '0006_157.6',
 '0006_158.5',
 '0006_159.6',
 '0006_159.8',
 '0006_160.3',
 '0006_173.4',
 '0006_173.7',
 '0006_174.0',
 '0006_174.4',
 '0006_174.9',
 '0006_175.6',
 '0006_176.1',
 '0006_177.0',
 '0006_177.2',
 '0006_177.5',
 '0006_177.9',
 '0006_178.8',
 '0006_181.6',
 '0006_184.1',
 '0006_187.5',
 '0006_194.8',
 '0006_203.6',
 '0006_211.2',
 '0006_216.2',
 '0006_221.1',
 '0007_000.0',
 '0007_000.1',
 '0007_000.3',
 '0007_001.5',
 '0007_002.4',
 '0007_003.4',
 '0007_005.1',
 '0007_006.1',
 '0007_007.3',
 '0007_008.3',
 '0007_009.3',
 '0007_010.5',
 '0007_012.5',
 '0007_015.0',
 '0008_000.0',
 '0008_000.4',
 '0008_001.2',
 '0009_000.0',
 '0009_000.5',
 '0009_001.1',
 '0009_002.8',
 '0009_004.9',
 '0009_005.4',
 '0009_006.6',
 '0009_007.0',
 '0009_007.5',
 '0009_007.9',
 '0009_008.3',
 '0009_008.6',
 '0009_009.1',
 '0009_009.6',
 '0009_009.8',
 '0009_010

Combine locked and unlocked


,SEGID,DY_VOL,FT,ATYPENAME,LANES,YEAR,SOURCE,SAID_FAC,FAC_FAL,FAC_SPR,FAC_WDAVG,FAC_SPRFAL,modAadt,aadtAdjFac,modForecast,CO_FIPS,SUBAREAID,inTdm
0,0013_000.0,13747.6,2.0,Suburban,4.0,2019.0,WFv901_Segments_20240226_Draft.shp,1.0,1.0243,1.0104,1.0924,1.01735,12585.0,6929.0,19500.0,NaN,NaN,NaN
1,0013_000.0,14506.3,2.0,Suburban,4.0,2023.0,WFv901_Segments_20240226_Draft.shp,1.0,1.0243,1.0104,1.0924,1.01735,13279.0,6929.0,20000.0,NaN,NaN,NaN
2,0013_000.0,15215.0,2.0,Suburban,4.0,2028.0,WFv901_Segments_20240226_Draft.shp,1.0,1.0243,1.0104,1.0924,1.01735,13928.0,6929.0,21000.0,NaN,NaN,NaN
3,0013_000.0,16248.9,2.0,Suburban,4.0,2032.0,WFv901_Segments_20240226_Draft.shp,1.0,1.0243,1.0104,1.0924,1.01735,14874.0,6929.0,22000.0,NaN,NaN,NaN
4,0013_000.0,15734.2,2.0,Suburban,4.0,2042.0,WFv901_Segments_20240226_Draft.shp,1.0,1.0243,1.0104,1.0924,1.01735,14403.0,6929.0,21500.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55792,WFRC_8476,1.0,0.0,0,0.0,2023.0,WF_Segments_20240326_Draft.shp,NaN,1.0243,1.0104,1.0924,1.01735,1.0,0.0,0.0,35.0,1.0,0.0
55793,WFRC_8476,1.0,0.0,0,0.0,2028.0,WF_Segments_20240326_Draft.shp,NaN,1.0243,1.0104,1.0924,1.01735,1.0,0.0,0.0,35.0,1.0,0.0
55794,WFRC_8476,1.0,0.0,0,0.0,2032.0,WF_Segments_20240326_Draft.shp,NaN,1.0243,1.0104,1.0924,1.01735,1.0,0.0,0.0,35.0,1.0,0.0
55795,WFRC_8476,1.0,0.0,0,0.0,2042.0,WF_Segments_20240326_Draft.shp,NaN,1.0243,1.0104,1.0924,1.01735,1.0,0.0,0.0,35.0,1.0,0.0


In [33]:
# export final filedfModWithAdjMerged
dfModWithAdjMerged.to_csv('intermediate/model-forecasts.csv', index=False)